# OMOP to Adverse Event

The mapping is to be found here: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=507968289#gid=507968289

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### Observation to Adverse Episode

In [ ]:
getAdverseEpisode="""
    SELECT person_id, observation_date, observation_concept_id, observation_source_concept_id FROM observation
    WHERE observation_concept_id IN (763790,765927,763791,765765,765766) AND observation_source_concept_id IN (35820298,763945,1704126,763944,35821988,35821987);
"""
getEpisodeEvent="""
    SELECT id FROM episode_event WHERE date_of_episode = %s AND cancer_episode IN (SELECT id FROM cancer_episode WHERE person_id = %s);
"""

insertAdverseEvent = """
    INSERT INTO adverse_event (adverse_event_duration, adverse_event_starting_date, adverse_event_type, episode_event)
    VALUES (%s, %s, %s, %s);
"""


curOMOP.execute(getAdverseEpisode)
adverseEvents = curOMOP.fetchall()

for event in adverseEvents:
    curIDEA.execute(getEpisodeEvent, (event[1], event[0]))
    episode = curIDEA.fetchone()
    if episode is not None:
        curOMOP.execute(insertAdverseEvent, (event[3], event[1], event[2], episode[0]))